Установим необходимые библиотеки для парсинга дискрипторов из библиотеки mordred. Поскольку библиотека mordred работает на основе rdkit импортируем и его.

In [ ]:
!pip install mordred
!pip install rdkit-pypi
from mordred import Calculator
from mordred import descriptors
from rdkit.Chem import AllChem
from rdkit import Chem
import pandas as pd
import numpy as np

С помощью библиотеки psndas считываем датасет.

In [3]:
data = pd.read_excel('7_whit_cid.xlsx')

In [4]:
#Выбираем колонку по которой будем парсить дискрипторы
smiles = data['CanonicalSMILES']
#Чтобы сократить время расчетов временно удалим все дублирующиеся smiles
smiles = smiles.drop_duplicates()
smiles = smiles.reset_index(drop = True)
#Удалим строчку с пустым занчением чтобы парсер не выдавал ошибку
smiles = smiles.drop(labels = [5],axis = 0)

Проверим правильно ли мы сформировали входные данные.

In [ ]:
smiles

Пишем цикл для париснга дискрипторов.

In [ ]:
#Создадим пустой датафрейм для записи туда дескрипторов
df = pd.DataFrame()
#В цикле проходимся по столбцусодержащим smiles
for i in smiles:  
  #В переменную забиваем калькулятор дескрипторов
  calc = Calculator(descriptors)
  #Получаем молекулу из smiles формата
  mol = [Chem.MolFromSmiles(i)]  
  #Расчитываем все дескрипторы для конкретной молекулы и заносим их в массив pandas
  a = calc.pandas(mol)
  #Добавляем все полученные значения в заранее созданный датафрейм
  df = pd.concat([df,a])

Проверяем правильно ли спарсились значения

In [11]:
df.to_excel('df.xlsx')

Для правильного объединения изначального датасета и полученных дескрипторов вставим колонку CanonicalSMILES в датафрейм с дескрипторами

In [ ]:
df = df.insert(0, "CanonicalSMILES", smiles)

Забиваем в переменную ту же колонку из изначального датасета но уже с дупликатами.

In [24]:
smiles_ = data['CanonicalSMILES']

При помощи функции merge востанавливаваем убранные ранее дубликаты. Для это используем left.join а индексной колонкой делаем общую колонку 'CanonicalSMILES'

In [28]:
mord_desk = pd.merge(smiles_, df, how='left', on='CanonicalSMILES')

Выгружаем полученный датасет, проверяем его и добавляем дескрипторы в изначальную

In [30]:
mord_desk.to_excel('mord_desk.xlsx')